# L5: 与任何大语言模型聊天! 💬

# L5: Chat with any LLM! 💬

在最后一课中，我们将构建一个与开源 LLM 聊天的应用程序。Falcon 40B，最好的开源模型之一。

我们将使用开源 LLM 构建一个聊天机器人应用程序。你可能已经用 ChatGPT 聊过天了，但运行它既费钱又死板。自定义 LLM 可以在本地运行，在自己的数据中进行微调，或者在云上运行，成本更低。在本课中，我们将使用运行 "falcon-40B-Instruct "的推理端点，它是最好的开源大型语言模型之一。

使用文本生成推理库在本地运行很方便。当然，也可以使用 Gradio 创建接口。Gradio 是基于 API 的 LLM，因此不仅支持开源的当然，你也可以使用 Gradio 为基于 API 的 LLM 创建接口，所以不仅仅是开源 LLM。但在本课程中，我们将重点介绍开源 LLM Falcon 40B。

加载 HF API 密钥和相关 Python 库

Load your HF API key and relevant Python libraries

In [1]:
import os
import io
import IPython.display
from PIL import Image
import base64 
import requests 
requests.adapters.DEFAULT_TIMEOUT = 60

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

我们在这里设置token和辅助函数。你可以看到，在这里我们使用了不同的库。我们使用的是文本生成库，这是一个用于处理开源 LLM 的精简库，可以让你同时加载 API（就像我们在这里做的一样），也可以在本地运行你自己的 LLM。

In [2]:
# Helper function
import requests, json
from text_generation import Client

#FalcomLM-instruct endpoint on the text_generation library
client = Client(os.environ['HF_API_FALCOM_BASE'], headers={"Authorization": f"Basic {hf_api_key}"}, timeout=120)

## 建立一个应用程序，与任何LLM聊天！

## Building an app to chat with any LLM!

这里我们将使用一个 [推理端点](https://huggingface.co/inference-endpoints) 来调用 `falcon-40b-instruct` ,  其在[🤗 开源LLM榜单](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)上名列前茅。

如果要本地调用它, 可以使用 [Transformers 库](https://huggingface.co/docs/transformers/index) 或者 [文本生成推理库](https://github.com/huggingface/text-generation-inference) 

Here we'll be using an [Inference Endpoint](https://huggingface.co/inference-endpoints) for `falcon-40b-instruct` , one of best ranking open source LLM on the [🤗 Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard). 

To run it locally, one can use the [Transformers library](https://huggingface.co/docs/transformers/index) or the [text-generation-inference](https://github.com/huggingface/text-generation-inference) 

In [3]:
prompt_chinese = "数学是被发明还是被发现的？"
client.generate(prompt_chinese, max_new_tokens=256).generated_text

'\n数学是被发现的，因为它是自然界的基本规律和模式。人类只是发现了这些规律和模式，并将它们用符号和公式表达出来。'

In [33]:
prompt = "Has math been invented or discovered?"
client.generate(prompt, max_new_tokens=256).generated_text

'\nMath has been both invented and discovered. It is a human invention in the sense that it is a system of rules and concepts that we have created to help us understand the world around us. However, it is also a discovery in the sense that it is a fundamental aspect of the universe that we have uncovered through our observations and experiments.'

在第 2 课中，我们使用了一个非常简单的 Gradio 界面，它有一个文本框输入和一个输出。在这里，我们也可以用类似的方式与 LLM 聊天。再次复制我们的prompt。在这里，我们可以决定需要多少token。这就是非常简单地向 LLM 提问的方法。但我们还是不能聊天，因为如果你再问一个后续问题，它就无法理解或保留上下文。

In [ ]:
# 回到第 2 课，时间过得真快！
#Back to Lesson 2, time flies!
import gradio as gr
def generate(input, slider):
    output = client.generate(input, max_new_tokens=slider).generated_text
    return output

demo = gr.Interface(fn=generate, inputs=[gr.Textbox(label="Prompt"), gr.Slider(label="Max new tokens", value=20,  maximum=1024, minimum=1)], outputs=[gr.Textbox(label="Completion")])
gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT1']))

因此，基本上我们要做的是，向模型发送我们之前的问题、它自己的回答以及后续问题。但建立所有这些都有点麻烦。这就是 Gradio 聊天机器人组件的作用所在，因为它允许我们简化向模型发送对话历史记录的过程。

因此，我们要解决这个问题。为此，我们将引入一个新的 Gradio 组件--Gradio Chatbot。

## 使用 `gr.Chatbot()` 来助力!

## `gr.Chatbot()` to the rescue!

让我们开始使用 Gradio Chatbot 组件。这里实例化了一个带有文本框提示和提交按钮的 Gradle ChatBot 组件，是一个非常简单的用户界面。但我们现在还不是在和 LLM 聊天。

只需随机选择三个预设回复，然后将我的信息和机器人的信息添加到聊天记录中。所以在这里，你可以看到我可以说任何话，它基本上会随机查看这三个回复。

In [ ]:
import random

def respond(message, chat_history):
        #这里没有 LLM，只需回复一条随机的预设信息
        #No LLM here, just respond with a random pre-made message
        bot_message_chinese = random.choice(["告诉我更多信息", 
                                     "酷，但我不感兴趣", 
                                     "嗯，那好吧"]) 
        bot_message = random.choice(["Tell me more about it", 
                                     "Cool, but I'm not interested", 
                                     "Hmmmm, ok then"]) 
        chat_history.append((message, bot_message_chinese))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #调整窗口大小 just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #点击enter来提交 Press enter to submit
gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT2']))

In [ ]:
"""
我们必须格式化聊天prompt。此处正在定义这个格式化聊天prompt函数。
在这里，我们要做的就是使其包含聊天历史记录，这样 LLM 就能知道上下文。
但这还不够。我们还需要告诉它，哪些信息来自用户，哪些信息来自 LLM 本身，也就是我们正在调用的助手。

因此，我们设置了格式聊天prompt功能，在聊天记录的每一轮中，都包含一条用户信息和一条助手信息，以便我们的模型能准确回答后续问题。
现在，我们要将格式化的prompt传递给我们的 API。
"""
def format_chat_prompt(message, chat_history):
    prompt = ""
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history):
        formatted_prompt = format_chat_prompt(message, chat_history)
        bot_message = client.generate(formatted_prompt,
                                     max_new_tokens=1024,
                                     stop_sequences=["\nUser:", "<|endoftext|>"]).generated_text
        chat_history.append((message, bot_message))
        return "", chat_history

"""
在这里，我们可以看到格式化prompt的外观。
现在，我们的聊天机器人应该可以回答后续问题了。
"""
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #点击enter来提交 Press enter to submit
gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT3']))

我们可以看到，我们向它发送了上下文。我们向它发送了信息，然后要求它完成。一旦我们进入另一个迭代循环，我们就会向它发送我们的整个上下文，然后要求它完成。这很酷。但是，如果我们一直这样迭代下去，那么模型在一次对话中所能接受的信息量就会达到极限，因为我们总是给它越来越多的之前对话的内容。

为了让模型发挥最大作用，我们可以在这里将最大token数`max_new_tokens`设置为 1024、 这是我们在 API 中运行的硬件条件下，该模型所能接受的最大值。

可以尝试以下prompt：
1. 哪些动物生活在热带草原？
2. 哪种动物最强壮？

因此，我们创建了一个简单但功能强大的用户界面，用于与LLM聊天。如果需要进一步Gradio 所能提供的最佳功能，我们可以创建一个包含更多功能的用户界面。

### 添加其他高级功能

### Adding other advanced features

In [ ]:
def format_chat_prompt(message, chat_history, instruction):
    prompt = f"System:{instruction}"
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history, instruction, temperature=0.7):
    prompt = format_chat_prompt(message, chat_history, instruction)
    chat_history = chat_history + [[message, ""]]
    stream = client.generate_stream(prompt,
                                      max_new_tokens=1024,
                                      stop_sequences=["\nUser:", "<|endoftext|>"],
                                      temperature=temperature)
                                      #停止序列 不生成用户答案 stop_sequences to not generate the user answer
    acc_text = ""
    #token流式化处理 Streaming the tokens
    for idx, response in enumerate(stream):
            text_token = response.token.text

            if response.details:
                return

            if idx == 0 and text_token.startswith(" "):
                text_token = text_token[1:]

            acc_text += text_token
            last_turn = list(chat_history.pop(-1))
            last_turn[-1] += acc_text
            chat_history = chat_history + [last_turn]
            yield "", chat_history
            acc_text = ""

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #调整窗口大小 just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    with gr.Accordion(label="Advanced options",open=False):
        system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
        """
        在这里，我们有高级选项，包括系统消息，它可以设置 LLM 与你聊天的模式。
        因此，在系统消息中，你可以说，例如，你是一个乐于助人的助手，或者你可以给它一个特定的语气，一个特定的语调，
        你希望它更有趣一点，更严肃一点，你真的可以玩弄系统消息提示，看看它对你的消息有什么影响。

        有些人甚至会想给 LLM 一个角色，比如你是一个提供法律建议的律师，或者你是一个提供医疗建议的医生，
        但要注意的是，众所周知，LLM 会以一种听起来很真实的方式提供虚假信息。
        因此，尽管使用Falcon 40B 进行实验和探索会很有趣，但在现实世界的应用场景中，必须为此类使用案例制定进一步的保障措施。
        """
        temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
        """
        还有其他高级参数，比如这里的温度。
        温度基本上就是你希望模型的变化程度。因此，如果将温度设为零，模型就会倾向于始终 对相同的输入做出相同的反应。
        所以同样的问题，同样的答案。温度越高，信息的变化就越多。但如果温度过高，它就会开始给出无意义的答案。
        因此，0.7 是一个不错的默认参数，但我们鼓励你多做尝试。
        """
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.queue().launch(share=True, server_port=int(os.environ['PORT4']))

除此之外，这个用户界面还能让我们进行流式传输回复。
它逐个token发送，我们可以看到它实时完成。因此，我们不需要等到整个答案都准备好了。在这里，我们可以看到它是如何完成的。如果你不理解这里的所有内容，也不用担心，因为我们的目的是用一个非常完整的用户界面来结束课程，并提供LLM方面的所有功能。

在格式聊天提示中，也就是我们之前使用的功能中，我们添加了一个新元素，那就是系统指令。因此，在开始用户助手对话之前，我们在系统顶部添加了一个指令。因此，基本上在发送给模型的每条信息的开头，都会有我们设置的系统信息。在这里，我们调用文本生成库的`generate_stream`函数。而 `generate_stream`函数的作用就是逐个生成响应标记。因此，在这个循环中，发生的事情就是按标记生成响应标记，将其添加到聊天记录中，然后将其返回给函数。

In [ ]:
gr.close_all()